In [136]:
#imports
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from nba_api.stats.endpoints import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time
import datetime
from datetime import datetime, timedelta
from nba_api.stats.library.http import NBAStatsHTTP
import joblib
from typing import Optional

# Set the timeout
NBAStatsHTTP.timeout = 120  # Set to 30 seconds; adjust as needed

In [27]:
def player_name(player_ID):
    players = playerindex.PlayerIndex()
    players_data = players.get_data_frames()[0]
    last_name = players_data.loc[players_data['PERSON_ID'] == player_ID, 'PLAYER_LAST_NAME'].iloc[0]
    first_name = players_data.loc[players_data['PERSON_ID'] == player_ID, 'PLAYER_FIRST_NAME'].iloc[0]
    player_Name = first_name + " " + last_name
    return player_Name
# print(player_name(1629008))
def player_id(player_Name):
    players = playerindex.PlayerIndex()
    name = player_Name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players_data = players.get_data_frames()[0]
    player_ID = players_data.loc[(players_data['PLAYER_FIRST_NAME'] == first_name) & (players_data['PLAYER_LAST_NAME'] == last_name), 'PERSON_ID'].iloc[0]
    return player_ID
# Example, should return 1629008
# player_id('Michael Porter Jr.')

In [28]:
def team_id(team_abbrev):
    teams = teamgamelogs.TeamGameLogs()
    teams_data = teams.get_data_frames()[0]
    team_id = teams_data.loc[teams_data['TEAM_ABBREVIATION'] == team_abbrev, 'TEAM_ID'].iloc[0]
    return team_id
# Example usage, should return: 1610612747
# print(team_id('LAL'))
def team_abbrev(team_ID):
    teams = teamdetails.TeamDetails(team_id= team_ID)
    teams_data = teams.get_data_frames()[0]
    team_abbrev = teams_data.loc[teams_data['TEAM_ID'] == team_ID, 'ABBREVIATION'].iloc[0]
    return team_abbrev
# team_abbrev(1610612747)

In [29]:
#function to get player gamelog based on Player Name rather than ID

def player_gamelog(player_name):
    id = player_id(player_name)
    gamelog = playergamelog.PlayerGameLog(player_id= id)
    exc = gamelog.get_data_frames()[0]
    return gamelog.get_data_frames()[0]
# player_gamelog('Michael Porter Jr.')


In [30]:
def playerVplayer(player_name, vs_player_name):
    id = player_id(player_name)
    vid = player_id(vs_player_name)
    gamelog = playervsplayer.PlayerVsPlayer(vs_player_id=vid, player_id=id)
    return gamelog.get_data_frames()

# playerVplayer('Nikola Jokic', 'Rudy Gobert')

In [ ]:
def getWins(team_id):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    return df['W'][0]
# print(getWins(team_id('BKN')))

def getWinsUpTo(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] <= game_date]
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['W']
# print(getWinsUpTo(team_id('WAS'), 'OCT 24, 2024'))
# print(getWinsUpTo(team_id('WAS')))
def getLosses(team_id):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    return df['L'][0]
# print(getLosses(team_id('WAS')))

def getLossesUpTo(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] <= game_date]
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['L']
# print(getLossesUpTo(team_id('OKC'), 'OCT 25, 2024'))
# print(getLossesUpTo(team_id('OKC')))

def getL5Wins(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] <= game_date]
    df_teamlog = df_teamlog.head(5)
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['W']
# print(getL5Wins(team_id('OKC')))


def getL5PPG(team_id, game_date: Optional[str] = None):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]
    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    if game_date:
        df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] <= game_date]
    df_teamlog = df_teamlog.head(5)
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['AVG_PTS']
# getL5PPGRev(team_id=team_id('OKC'), game_date= 'OCT 24, 2024')

0
6


In [32]:
def age(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_age = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'AGE'].iloc[0]
    return player_age
# age(player_id('Luka Dončić'))

In [33]:
def height(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_height = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_HEIGHT_INCHES'].iloc[0]
    return player_height
# Example, should return 83(in)
# height(player_id('Nikola Jokić'))

In [34]:
def weight(player_ID):
    players = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
    players_data = players.get_data_frames()[0]
    player_weight = players_data.loc[players_data['PLAYER_ID'] == player_ID, 'PLAYER_WEIGHT'].iloc[0]
    return player_weight
# weight(player_id('Luka Dončić'))

In [35]:
def encode_positions(position):
    roles = {'Primary_Guard': 0, 'Primary_Forward': 0, 'Primary_Center': 0, 'Is_Guard': 0, 'Is_Forward': 0, 'Is_Center': 0}
    primary = position.split('-')[0]
    if 'G' == primary:
        roles['Primary_Guard'] = 1
    elif 'F' == primary:
        roles['Primary_Forward'] = 1
    elif 'C' == primary:
        roles['Primary_Center'] = 1
    if 'G' in position:
        roles['Is_Guard'] = 1
    if 'F' in position:
        roles['Is_Forward'] = 1
    if 'C' in position:
        roles['Is_Center'] = 1
    return pd.Series(roles)
# print(encode_positions('C-G'))
def position(player_ID):
    player_Name = player_name(player_ID)
    name = player_Name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    position = df.loc[(df['PLAYER_LAST_NAME'] == last_name) & (df['PLAYER_FIRST_NAME'] == first_name), 'POSITION'].iloc[0]
    return position 
# position(1629008) 
# "LeBron, F"

In [36]:
def player_team(player_name):
    name = player_name.split()
    first_name = name[0]
    last_name = " ".join(name[1:])
    players = playerindex.PlayerIndex()
    df = players.get_data_frames()[0]
    df.to_excel('Player_Index.xlsx')
    team_ID = df.loc[(df['PLAYER_LAST_NAME'] == last_name) & (df['PLAYER_FIRST_NAME'] == first_name), 'TEAM_ID'].iloc[0]
    team = team_abbrev(team_ID)
    return team
# player_team('Collin Sexton')

In [37]:
# Will add a Home_Away column to the gamelog so that the ML model can use that as a variable
def homeOrAway(player_gamelog):
    player_gamelog['Home_Away'] = player_gamelog['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    return player_gamelog
# kingStats = player_gamelog('LeBron James')
# homeOrAway

In [38]:
def extract_team(matchup):
     # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[0]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[0]
    else:
        return "Invalid Matchup Format"
# print(extract_team('LAL @ NYK'))

def extract_opponent(matchup):
    # The opponent's team abbreviation is after "vs. " or "@ "
    if " vs. " in matchup:
        # Split the string at " vs. " and take the second part
        return matchup.split(" vs. ")[1]
    elif " @ " in matchup:
        # Split the string at " @ " and take the second part
        return matchup.split(" @ ")[1]
    else:
        return "Invalid Matchup Format"
# extract_opponent('LAL @ NYK')

In [39]:
def againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev, game_date):
    df_copy = player_gamelog.copy()
    df_copy['GAME_DATE'] = pd.to_datetime(df_copy['GAME_DATE'])
    game_date = pd.to_datetime(game_date)
    awaySearchString = home_team_abbrev + ' vs. ' + opp_team_abbrev
    homeSearchString = home_team_abbrev + ' @ ' + opp_team_abbrev
    df_copy = df_copy.iloc[::-1].reset_index(drop=True)
    filtered_log = df_copy[(df_copy['GAME_DATE'] < game_date)]
    newGameLog = filtered_log.loc[filtered_log.MATCHUP.isin([homeSearchString, awaySearchString])]
    newGameLog = newGameLog.sort_values(by='GAME_DATE', ascending=True)
    newGameLog['GAME_DATE'] = newGameLog['GAME_DATE'].apply(lambda x: x.strftime('%b').upper() + x.strftime(' %d, %Y'))
    return newGameLog

# Test
# kingStats = player_gamelog('LeBron James')
# print(againstThisTeam(kingStats, 'LAL', 'SAS', 'JAN 14, 2025'))

def boxStatsAgainstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev):
    new_log = againstThisTeam(player_gamelog, home_team_abbrev, opp_team_abbrev)
    new_log = new_log.loc[:, ['GAME_DATE', 'MATCHUP', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'MIN']]
    return new_log
# test
# dfoxStats = player_gamelog("De'Aaron Fox")
# boxStatsAgainstThisTeam(dfoxStats, 'SAC', 'MEM')

In [40]:
def fillMatchupStats(player_log):
    stats_to_average = ['PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'OREB', 'STL', 'MIN', 'PLUS_MINUS'] 

    for i in range(player_log.shape[0]):
        team_abbrev = extract_team(player_log["MATCHUP"][i])
        opp_team_abbrev = extract_opponent(player_log["MATCHUP"][i])
        game_date = player_log['GAME_DATE'].iloc[i]
        matchup_stats = againstThisTeam(player_log, team_abbrev, opp_team_abbrev, game_date)
        if matchup_stats.empty:
            for stat in stats_to_average:
                player_log[f'VS_AVG_{stat}'][i] = player_log[f'AVG_{stat}'][i]
        else:
            for stat in stats_to_average:
                player_log.loc[i, f'VS_AVG_{stat}'] = matchup_stats[stat].mean()
    return player_log

# df = pd.read_csv('v2.2 csvs/Alperen Sengun.csv')
# fillMatchupStats(df)

In [274]:
def getOFFefficiency(team_ID):
    offenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = offenses.get_data_frames()[0]
    offEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_OFF_RATING'].iloc[0]
    return offEfficiency
# print(getOFFefficiency(team_id('CLE')))
def getPPG(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['PointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# print(getPPG(team_id('LAL')))
def getPPGUpTo(team_id, date):
    teamlog = teamgamelog.TeamGameLog(team_id= team_id)
    df = teamlog.get_data_frames()[0]
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='%b %d, %Y')
    target_date = pd.to_datetime(date, format='%b %d, %Y')

    filtered_df = df[df['GAME_DATE'] <= target_date]
    sum_points = filtered_df['PTS'].sum()
    return sum_points / len(filtered_df)
def getPPGUpTo(team_id, date):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]

    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    date = pd.to_datetime(date)
    df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] <= date]
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)

    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[0]['AVG_PTS']
# print(getPPGUpTo(team_id('HOU'), 'JAN 21, 2025'))

In [275]:
# Note: the numbers in the def efficiency do not match the numbers on the official website, yet they are in the correct rankings, (i.e OKC's def efficiency is #1 in this data, and #1 on the official data, however they have differing def efficiency numbers, one could be 102.9 the other 103.1)
def getDEFefficiency(team_ID):
    defenses = teamestimatedmetrics.TeamEstimatedMetrics()
    df = defenses.get_data_frames()[0]
    defEfficiency = df.loc[df['TEAM_ID'] == team_ID, 'E_DEF_RATING'].iloc[0]
    return defEfficiency
# print(getDEFefficiency(team_id('LAL')))
    
def getPPG_allowed(team_ID):
    team_stats = leaguestandingsv3.LeagueStandingsV3()
    df_teamstats = team_stats.get_data_frames()[0]
    team_row = df_teamstats[df_teamstats['TeamID'] == team_ID]
    if not team_row.empty:
        totalPTSA = team_row.iloc[0]['OppPointsPG']
        return totalPTSA
    else:
        print("No data found for the specified city")
        return None
# Example Usage, Output: 113.70588235294117
# print(getPPG_allowed(team_id('OKC')))

def getPPG_allowedUpTo(team_id, date):
    teamlog = teamgamelog.TeamGameLog(team_id)
    df_teamlog = teamlog.get_data_frames()[0]

    df_teamlog['GAME_DATE'] = pd.to_datetime(df_teamlog['GAME_DATE'])
    date = pd.to_datetime(date)
    df_teamlog = df_teamlog[df_teamlog['GAME_DATE'] <= date]
    game_ids = df_teamlog['Game_ID'].tolist()
    game_ids_string = '|'.join(game_ids)

    cume = cumestatsteam.CumeStatsTeam(team_id, game_ids_string)
    df = cume.get_data_frames()[1]
    return df.iloc[1]['AVG_PTS']
# getPPG_allowedUpTo(team_id('OKC'), 'NOV 10, 2024')


In [239]:
#players to add: "CJ McCollum", "Zion Williamson", "De'Andre Hunter", "Andrew Wiggins", "Johnathan Kuminga", "Keyonte George", "Bennedict Mathurin", "Jalen Suggs", "Malik Beasley", "Gradey Dick", "OG Anunoby", "Jimmy Butler", "Derrick White", "Bam Adebayo", "Myles Turner", "Fred VanVleet", "Jerami Grant", "Ivica Zubac", "Dennis Schröder", "Jakob Poeltl", "Payton Pritchard", "Deni Avdija", "Christian Braun", "Josh Hart", "Jarrett Allen", "Deandre Ayton", "P.J. Washington", "Kelly Oubre Jr.", "Naz Reid", "Klay Thompson", "Dyson Daniels", "Bobby Portis", "Russell Westbrook", "Tobias Harris", "Amen Thompson", "D'Angelo Russell", "Dillon Brooks", "Santi Aldama", "Ayo Dosunmu", "Bilal Coulibaly", "Keldon Johnson", "Brook Lopez", "Daniel Gafford", "Rui Hachimura", "Buddy Hield", "Scoot Henderson", "Jabari Smith Jr.", "Keegan Murray", "Ty Jerome"

playerList = [
        "Alperen Sengun", "Anfernee Simons", "Anthony Davis", "Anthony Edwards", "Austin Reaves", "Brandon Miller", "Bradley Beal", "Cade Cunningham", "Cameron Johnson", "Coby White", "Collin Sexton", "Damian Lillard", "Darius Garland", "De'Aaron Fox", "DeMar DeRozan", "Desmond Bane", "Devin Booker", "Domantas Sabonis", "Donovan Mitchell", "Evan Mobley", "Giannis Antetokounmpo", "Jalen Brunson", "Jalen Green", "Jalen Johnson", "Jalen Williams", "Jamal Murray", "James Harden", "Jaren Jackson Jr.", "Jaylen Brown", "Jayson Tatum", "John Collins", "Jordan Poole", "Julius Randle", "Karl-Anthony Towns", "Kevin Durant", "Kristaps Porziņģis", "Kyrie Irving", "LaMelo Ball", "Lauri Markkanen", "LeBron James", "Luka Dončić", "Malik Monk", "Michael Porter Jr.", "Mikal Bridges", "Miles Bridges", "Nikola Jokić", "Nikola Vučević", "Norman Powell", "Pascal Siakam", "Paul George", "RJ Barrett", "Shaedon Sharpe", "Shai Gilgeous-Alexander", "Stephen Curry", "Trae Young", "Tyler Herro", "Tyrese Haliburton", "Tyrese Maxey", "Victor Wembanyama", "Zach LaVine"
    ]
def createPlayerTeamDict():
    playerTeamDict = {}
    for player in playerList:
        team = player_team(player)  # Get the team for this player
        playerTeamDict[player] = team
    return playerTeamDict
# createPlayerTeamDict()
playerTeamDict = {'Alperen Sengun': 'HOU', 'Anfernee Simons': 'POR', 'Anthony Davis': 'LAL', 'Anthony Edwards': 'MIN', 'Austin Reaves': 'LAL', 'Brandon Miller': 'CHA', 'Bradley Beal': 'PHX', 'Cade Cunningham': 'DET', 'Cameron Johnson': 'BKN', 'Coby White': 'CHI', 'Collin Sexton': 'UTA', 'Damian Lillard': 'MIL', 'Darius Garland': 'CLE', "De'Aaron Fox": 'SAC', 'DeMar DeRozan': 'SAC', 'Desmond Bane': 'MEM', 'Devin Booker': 'PHX', 'Domantas Sabonis': 'SAC', 'Donovan Mitchell': 'CLE', 'Evan Mobley': 'CLE', 'Giannis Antetokounmpo': 'MIL', 'Jalen Brunson': 'NYK', 'Jalen Green': 'HOU', 'Jalen Johnson': 'ATL', 'Jalen Williams': 'OKC', 'Jamal Murray': 'DEN', 'James Harden': 'LAC', 'Jaren Jackson Jr.': 'MEM', 'Jaylen Brown': 'BOS', 'Jayson Tatum': 'BOS', 'John Collins': 'UTA', 'Jordan Poole': 'WAS', 'Julius Randle': 'MIN', 'Karl-Anthony Towns': 'NYK', 'Kevin Durant': 'PHX', 'Kristaps Porziņģis': 'BOS', 'Kyrie Irving': 'DAL', 'LaMelo Ball': 'CHA', 'Lauri Markkanen': 'UTA', 'LeBron James': 'LAL', 'Luka Dončić': 'DAL', 'Malik Monk': 'SAC', 'Michael Porter Jr.': 'DEN', 'Mikal Bridges': 'NYK', 'Miles Bridges': 'CHA', 'Nikola Jokić': 'DEN', 'Nikola Vučević': 'CHI', 'Norman Powell': 'LAC', 'Pascal Siakam': 'IND', 'Paul George': 'PHI', 'RJ Barrett': 'TOR', 'Shaedon Sharpe': 'POR', 'Shai Gilgeous-Alexander': 'OKC', 'Stephen Curry': 'GSW', 'Trae Young': 'ATL', 'Tyler Herro': 'MIA', 'Tyrese Haliburton': 'IND', 'Tyrese Maxey': 'PHI', 'Victor Wembanyama': 'SAS', 'Zach LaVine': 'CHI'}
def playedYesterday():
    tlog = leaguegamelog.LeagueGameLog()
    df = tlog.get_data_frames()[0]
    yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    yesterdayGames = df[df['GAME_DATE'] == yesterday]
    yesterdayTeams = yesterdayGames['TEAM_ABBREVIATION'].tolist()
    
    played = []
    for player in playerList:
        team = playerTeamDict.get(player)  # Get the team of the player
        if team in yesterdayTeams:
            played.append(player)
    
    return played
# playedYesterday()
def getPlayerTeam(player_id):
    return playerTeamDict.get(player_name(player_id))
# getPlayerTeam(player_id('Paul George'))

In [450]:
def last5ID(player_id, date):
    date = pd.to_datetime(date)

    df = player_gamelog(player_name(player_id))
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    df = df[df['GAME_DATE'] <= date]
    df = df['Game_ID']
    df = df.head(5)

    teamlog = teamgamelog.TeamGameLog(team_id= team_id(getPlayerTeam(player_id)))
    df_team = teamlog.get_data_frames()[0]
    df_team['GAME_DATE'] = pd.to_datetime(df_team['GAME_DATE'])
    df_team = df_team[df_team['GAME_DATE'] <= date]
    df_team = df_team['Game_ID']
    df_team = df_team.head(5)
    
    common_games = pd.merge(df, df_team, on='Game_ID', how='inner')
    return common_games['Game_ID'].tolist()
# print(last5ID(player_id('Shai Gilgeous-Alexander'),  date='JAN 19, 2025'))
def last5Stats(player_id, date):
    game_ids = last5ID(player_id, date)
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsplayer.CumeStatsPlayer(player_id, game_ids_string)
    df = cume.get_data_frames()[1]
    stats_to_average = ['FG', 'FGA', 'FG3', 'FG3A', 'OFF_REB', 'FT', 'FTA']
    for stat in stats_to_average:
        avg_column_name = f'AVG_{stat}'
        df[avg_column_name] = df[stat] / df['GP']

    df['AVG_MIN'] = df['AVG_ACTUAL_MINUTES'] + (df['AVG_ACTUAL_SECONDS'] / 60.0)
    df.rename(columns={'AVG_FG': 'AVG_FGM', 'AVG_FG3': 'AVG_FG3M', 'AVG_OFF_REB': 'AVG_OREB', 'AVG_FT': 'AVG_FTM', 'FT_PCT': 'AVG_FT_PCT', 'FG_PCT': 'AVG_FG_PCT', 'FG3_PCT': 'AVG_FG3_PCT'}, inplace=True)
    return df
# print(last5Stats(player_id('Shai Gilgeous-Alexander'),  date='OCT 30, 2024'))
def allPlayerGameIDS(player_id, game_date: Optional[str] = None):
    cume = cumestatsplayergames.CumeStatsPlayerGames(player_id)
    df = cume.get_data_frames()[0]

    teamlog = teamgamelog.TeamGameLog(team_id= team_id(getPlayerTeam(player_id)))
    df_team = teamlog.get_data_frames()[0]
    if game_date:
        df_team['GAME_DATE'] = pd.to_datetime(df_team['GAME_DATE'])
        game_date = pd.to_datetime(game_date)
        df_team = df_team[df_team['GAME_DATE'] <= game_date]
    df_team.rename(columns={'Game_ID': 'GAME_ID'}, inplace=True)
    df_team = df_team['GAME_ID']
    df = df['GAME_ID']

    common_games = pd.merge(df, df_team, on='GAME_ID', how='inner')
    return common_games['GAME_ID'].tolist()
# allPlayerGameIDS(player_id('Donovan Mitchell'), 'OCT 25, 2024')
def seasonAverages(player_id, game_date: Optional[str] = None):
    # Can be up to a certain date or just overall averages
    game_ids = allPlayerGameIDS(player_id, game_date)
    game_ids_string = '|'.join(game_ids)
    cume = cumestatsplayer.CumeStatsPlayer(player_id, game_ids_string)
    df = cume.get_data_frames()[1]
    stats_to_average = ['FG', 'FGA', 'FG3', 'FG3A', 'OFF_REB', 'FT', 'FTA']
    for stat in stats_to_average:
        avg_column_name = f'AVG_{stat}'
        df[avg_column_name] = df[stat] / df['GP']

    df['AVG_MIN'] = df['AVG_ACTUAL_MINUTES'] + (df['AVG_ACTUAL_SECONDS'] / 60.0)
    df.rename(columns={'AVG_FG': 'AVG_FGM', 'AVG_FG3': 'AVG_FG3M', 'AVG_OFF_REB': 'AVG_OREB', 'AVG_FT': 'AVG_FTM', 'FT_PCT': 'AVG_FT_PCT', 'FG_PCT': 'AVG_FG_PCT', 'FG3_PCT': 'AVG_FG3_PCT'}, inplace=True)
    return df
# seasonAverages(player_id('Shai Gilgeous-Alexander')).to_csv('SGA season avg.csv')


In [44]:
def concatAllCSVS(folder):
    # List to hold dataframes
    dfs = []
    
    # Iterate over each file in the directory
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            # Construct full file path
            file_path = os.path.join(folder, filename)
            # Read the CSV file and append to list
            df = pd.read_csv(file_path, index_col=0)
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            df = df.iloc[7:].reset_index(drop=True)
            average_points = df['PTS'].mean()
            df['target'] = (df['PTS'] > average_points).astype(int)
            df.dropna(inplace=True)
            dfs.append(df)
    
    # Concatenate all dataframes in the list
    full_df = pd.concat(dfs, ignore_index=True)
    return full_df
# master_csv = concatAllCSVS('players csvs')

In [45]:
# THIS IS TO CREATE A MODEL WITH EVERYONE'S CSVS CONCATTED
def createModelALL():
    file_path = 'masterCSV_2025-01-22 v2.2.csv'  # Specify your CSV file name here
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)
    # Calculate average points
    average_points = df['PTS'].mean()
    df['target'] = (df['PTS'] > average_points).astype(int)

    # Features including PLAYER_ID
    categorical_features = ['MATCHUP', 'Player_ID', 'TEAM_ID', 'OPP_TEAMID']
    classifier = Pipeline([
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
            remainder='passthrough')),
        ('classifier', LogisticRegression(solver = 'liblinear', max_iter=50000, warm_start=True))
    ])
    # 'TEAM_ID', 'OPP_TEAMID',
    X = df[['MATCHUP', 'Player_ID', 'TEAM_ID', 'OPP_TEAMID', 'MA_PTS', 'MA_FGM', 'MA_FGA', 'MA_STL', 'MA_MIN', 'MA_PLUS_MINUS', 'AVG_PTS', 'MA_FG_PCT', 'OPP_PPG_ALLOWED', 'PPG', 'Home_Away', 'HEIGHT', 'WEIGHT', 'AGE', 'MA_TOV', 'MA_BLK', 'Primary_Guard', 'Primary_Forward','Primary_Center' , 'Is_Guard', 'Is_Forward', 'Is_Center', 'TEAM_W', 'TEAM_L', 'L5_W', 'L5_L', 'L5_PPG', 'OPP_TEAM_W', 'OPP_TEAM_L', 'OPP_L5_W', 'OPP_L5_L', 'OPP_L5_PPG']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    # Save the model
    joblib.dump(classifier, 'master_model_libV2.2.pkl')
# createModelALL()

In [46]:
# NOTE!!!!!!!!!!!! THIS SHOULD ONLY BE USED ONCE TO CREATE A NEW CSV FOR A PLAYER. ONCE A CSV HAS BEEN CREATED, USE THE updateStats METHOD TO UPDATE THEIR MOST CURRENT GAME.

def importantStatsRev(player_name):
    player_gameLog = player_gamelog(player_name)
    player_gameLog = player_gameLog.iloc[::-1].reset_index(drop=True)
    player_gameLog = player_gameLog[player_gameLog['MIN'] >= 20]
    window_size = 5
    stats_to_average = ['PTS', 'FGM', 'FGA', 'STL', 'BLK', 'TOV', 'FG_PCT', 'MIN', 'PLUS_MINUS']
    for stat in stats_to_average:
        player_gameLog[f'MA_{stat}'] = player_gameLog[stat].rolling(window=window_size, min_periods=1).mean().shift(0)
    player_gameLog['AVG_PTS'] = player_gameLog['PTS'].expanding().mean().shift(1)

    # Extract opponent team abbreviation
    player_gameLog['TEAM_ABBREVIATION'] = player_gameLog['MATCHUP'].apply(extract_team)
    player_gameLog['TEAM_ID'] = player_gameLog['TEAM_ABBREVIATION'].apply(team_id)
    player_gameLog['OPP_TEAM'] = player_gameLog['MATCHUP'].apply(extract_opponent)
    player_gameLog['OPP_TEAMID'] = player_gameLog['OPP_TEAM'].apply(team_id)
    # Fetch the PPG allowed by the opponent team
    player_gameLog['OPP_PPG_ALLOWED'] = player_gameLog['OPP_TEAMID'].apply(getPPG_allowed)
    player_gameLog['OPP_DEF_EFFICIENCY'] = player_gameLog['OPP_TEAMID'].apply(getDEFefficiency)
    player_gameLog['PPG'] = player_gameLog['TEAM_ID'].apply(getPPG)
    player_gameLog['OFF_EFFICIENCY'] = player_gameLog['TEAM_ID'].apply(getOFFefficiency)
    player_gameLog['POSITION'] = player_gameLog['Player_ID'].apply(position)
    encoded_positions = player_gameLog['POSITION'].apply(encode_positions)
    player_gameLog = pd.concat([player_gameLog, encoded_positions], axis=1)
    player_gameLog['AGE'] = player_gameLog['Player_ID'].apply(age)
    player_gameLog['HEIGHT'] = player_gameLog['Player_ID'].apply(height)
    player_gameLog['WEIGHT'] = player_gameLog['Player_ID'].apply(weight)
    player_gameLog['TEAM_W'] = player_gameLog['TEAM_ID'].apply(getWins)
    player_gameLog['TEAM_L'] = player_gameLog['TEAM_ID'].apply(getLosses)
    player_gameLog['L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['L5_L'] = 5 - player_gameLog['L5_W']
    player_gameLog['L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_TEAM_W'] = player_gameLog['OPP_TEAMID'].apply(getWins)
    player_gameLog['OPP_TEAM_L'] = player_gameLog['OPP_TEAMID'].apply(getLosses)
    player_gameLog['OPP_L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_L5_L'] = 5 - player_gameLog['L5_W']
    player_gameLog['OPP_L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)

    # Reset the index if needed (optional, for cleaner indexing)
    player_gameLog.reset_index(drop=True, inplace=True)
    homeOrAway(player_gameLog)
    return player_gameLog
# Example Usage, runtime ~1m 18s
players_list_all = [
    
]
# for player in players_list_all:
#     file_path = os.path.join('v2.1 csvs', player + '.csv')
#     df_updated = importantStatsRev(player)
#     df_updated.to_csv(file_path)
#     print("This player's csv was created: ", player)
# masterTESTcsv = concatAllCSVS('v2.1 csvs')
# masterTESTcsv.to_csv('masterCSV_2025-01-20 v2.1 newer.csv')

In [47]:
# USE THIS METHOD ONCE A CSV HAS BEEN CREATED FOR A PLAYER
def updateStatsRev(player_csv, player_name):
    df = pd.read_csv(player_csv)
    df_new = importantStatsRev(player_name)
    last_row = df_new.tail(1)
    df = pd.concat([df, last_row])
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    if df.iloc[-1]['GAME_DATE'] == df.iloc[-2]['GAME_DATE']:
        df = df.iloc[:-1]
        print(f"This {player_name} CSV is up to date")
    else:
        print(f"Updated {player_name} CSV")
        # file_path = os.path.join('test csvs', player_name + '.csv')
        # df.to_csv(file_path, index=False)
    return df

playedList = playedYesterday()
playedList = playedList[12::]
print(playedList)

def updateAllRev(player_list, folder):
    for player in player_list:
        file_path = os.path.join(folder, player + '.csv')
        df_updated = updateStatsRev(file_path, player)
        df_updated.to_csv(file_path)
# updateAllRev(playedList, 'v2.1 csvs')
# df = concatAllCSVS('v2.1 csvs')
# today = datetime.today().strftime('%Y-%m-%d')
# file_name = f"masterCSV_{today} v2.1.csv"
# df.to_csv(file_name, index=False)

['Domantas Sabonis', 'Donovan Mitchell', 'Evan Mobley', 'Jalen Green', 'Jalen Johnson', 'Jalen Williams', 'James Harden', 'Jaren Jackson Jr.', 'Jaylen Brown', 'Jayson Tatum', 'John Collins', 'Julius Randle', 'Kevin Durant', 'Kristaps Porziņģis', 'Kyrie Irving', 'LaMelo Ball', 'Lauri Markkanen', 'Malik Monk', 'Norman Powell', 'Shai Gilgeous-Alexander', 'Stephen Curry', 'Trae Young']


In [414]:
def testNewAccuracy(iterations=500):
    # Load the dataset
    df = pd.read_csv('masterCSV_2025-01-25 v2.3.csv')

    # Define features and target, 
    X = df[['Player_ID','MATCHUP','TEAM_ID','OPP_TEAMID','Home_Away','Primary_Guard','Primary_Forward','Primary_Center','Is_Guard','Is_Forward','Is_Center','AGE','HEIGHT','WEIGHT','TEAM_W','TEAM_L','PPG','L5_W','L5_L','L5_PPG','OPP_TEAM_W','OPP_TEAM_L','OPP_PPG_ALLOWED','OPP_L5_W','OPP_L5_L','OPP_L5_PPG','MA_PTS','MA_FGM','MA_FGA','MA_FG_PCT','MA_FG3M','MA_FG3A','MA_FG3_PCT','MA_OREB','MA_STL','MA_MIN','MA_PLUS_MINUS','AVG_PTS','AVG_FGM','AVG_FGA','AVG_FG_PCT','AVG_FG3M','AVG_FG3A','AVG_FG3_PCT','AVG_FTM','AVG_FTA','AVG_FT_PCT','AVG_OREB','AVG_STL','AVG_MIN','AVG_PLUS_MINUS','VS_AVG_PTS','VS_AVG_FGM','VS_AVG_FGA','VS_AVG_FG_PCT','VS_AVG_FG3M','VS_AVG_FG3A','VS_AVG_FG3_PCT','VS_AVG_OREB','VS_AVG_STL','VS_AVG_MIN','VS_AVG_PLUS_MINUS']]
    y = df['target']

    # Load the trained model
    model = joblib.load('master_model_lib v2.3.pkl')

    # List to store accuracies
    accuracies = []

    # Run the testing process multiple times
    for i in range(iterations):
        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=20, random_state=i)  # Vary random_state for different splits

        # Predict and calculate accuracy
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        accuracies.append(accuracy)

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracies)
    print(f"\nAverage Accuracy over {iterations} iterations: {average_accuracy:.4f}")

    lowest = min(accuracies)
    highest = max(accuracies)
    print("Lowest accuracy:", lowest)
    print("Highest accuracy:", highest)

# Run the function with the desired number of iterations
testNewAccuracy(iterations=500)


Average Accuracy over 500 iterations: 0.7952
Lowest accuracy: 0.55
Highest accuracy: 1.0


In [121]:
def warm_start_train(old_model_path, data, output_model_path):
    """
    Train LogisticRegression with warm start using combined old and new data.

    Parameters:
    - old_model_path: Path to the previously saved model.
    - new_data_path: Path to the new data CSV.
    - master_csv_path: Path to the master dataset CSV.
    - output_model_path: Path to save the updated model.
    """
    # Load new data
    new_data = pd.read_csv(data)
    new_data = new_data.iloc[:, 1:] 
    new_data = new_data.iloc[7:].reset_index(drop=True)
    # Define features and target
    X = new_data[['Player_ID','MATCHUP','TEAM_ID','OPP_TEAMID','Home_Away','Primary_Guard','Primary_Forward','Primary_Center','Is_Guard','Is_Forward','Is_Center','AGE','HEIGHT','WEIGHT','TEAM_W','TEAM_L','PPG','L5_W','L5_L','L5_PPG','OPP_TEAM_W','OPP_TEAM_L','OPP_PPG_ALLOWED','OPP_L5_W','OPP_L5_L','OPP_L5_PPG','MA_PTS','MA_FGM','MA_FGA','MA_FG_PCT','MA_FG3M','MA_FG3A','MA_FG3_PCT','MA_OREB','MA_STL','MA_MIN','MA_PLUS_MINUS','AVG_PTS','AVG_FGM','AVG_FGA','AVG_FG_PCT','AVG_FG3M','AVG_FG3A','AVG_FG3_PCT','AVG_FTM','AVG_FTA','AVG_FT_PCT','AVG_OREB','AVG_STL','AVG_MIN','AVG_PLUS_MINUS','VS_AVG_PTS','VS_AVG_FGM','VS_AVG_FGA','VS_AVG_FG_PCT','VS_AVG_FG3M','VS_AVG_FG3A','VS_AVG_FG3_PCT','VS_AVG_OREB','VS_AVG_STL','VS_AVG_MIN','VS_AVG_PLUS_MINUS']]
    y = new_data['target']
    # Train the model on the combined data
    model = joblib.load(old_model_path)
    model.fit(X, y)
    print("Model training completed.")

    # Save the updated model
    joblib.dump(model, output_model_path)
    print(f"Updated model saved to {output_model_path}")

# Example usage
warm_start_train(
    old_model_path='master_model_lib v2.3.pkl', 
    data = 'masterCSV_2025-01-25 v2.3.csv',
    output_model_path='master_model_lib v2.3 warm.pkl'
)


Model training completed.
Updated model saved to master_model_lib v2.3 warm.pkl


In [398]:
def getPredictData(player_name, matchup):
    # Construct the file path using the player's name
    file_path = f'v2.3 csvs/{player_name}.csv' 
    df = pd.read_csv(file_path)
    df = df.iloc[:, 1:] 
    df = df.iloc[7:].reset_index(drop=True)
    df = df.tail(1)
    if ' @ ' in matchup:
        teams = matchup.split(' @ ')
        home_away = 0  
    elif ' vs. ' in matchup:
        teams = matchup.split(' vs. ')
        home_away = 1 
    # features you need to manually update:
    # 'MATCHUP', 'OPP_PPG_ALLOWED', 'OPP_DEF_EFFICIENCY'(if included), 'Home_Away', 'OPP_TEAM_W', 'OPP_TEAM_L', 'OPP_L5_W', 'OPP_L5_L', 'OPP_L5_PPG'
    #'VS_AVG_PTS','VS_AVG_FGM','VS_AVG_FGA','VS_AVG_FG_PCT','VS_AVG_FG3M','VS_AVG_FG3A','VS_AVG_FG3_PCT','VS_AVG_FTM','VS_AVG_FTA','VS_AVG_FT_PCT','VS_AVG_OREB','VS_AVG_STL','VS_AVG_MIN','VS_AVG_PLUS_MINUS'
    df['MATCHUP'] = matchup
    df['OPP_TEAM'] = extract_opponent(matchup)
    df['OPP_PPG_ALLOWED'] = getPPG_allowed(team_id(teams[1]))
    df['OPP_DEF_EFFICIENCY'] = getDEFefficiency(team_id(teams[1]))
    df['OPP_TEAM_W'] = getWins(team_id(teams[1]))
    df['OPP_TEAM_L'] = getLosses(team_id(teams[1]))
    df['OPP_L5_W'] = getL5Wins(team_id(teams[1]))
    df['OPP_L5_L'] = 5 - df['OPP_L5_W']
    df['OPP_L5_PPG'] = getL5PPG(team_id(teams[1]))
    df['Home_Away'] = home_away
    df['PPG'] = getPPG(team_id(teams[0]))
    # today = datetime.date.today()
    # formatted_date = today.strftime('%b %d, %Y').upper()
    againstThisTeam(df, df['TEAM_ABBREVIATION'], teams[1], 'JAN 25, 2025')
    return df

def predict_performance(new_data):
    classifier = joblib.load('master_model_lib v2.3.pkl')
    
    predictions = classifier.predict(new_data)
    probabilities = classifier.predict_proba(new_data)
    
    return predictions, probabilities

# Example usage
# Assuming 'new_data' is DataFrame with the same features used in training
new_data = getPredictData("James Harden", 'LAC vs. MIL')
performance_predictions, performance_probabilities = predict_performance(new_data)
print("Line: " + str(new_data['AVG_PTS'].values[0]) + " PTS")
print("Predictions:", performance_predictions)  
print("Probabilities:", performance_probabilities)

Line: 21.325 PTS
Predictions: [0]
Probabilities: [[0.92940353 0.07059647]]


In [399]:
def kelly_criterion(bankroll, vegas_odds, probability):
    if vegas_odds > 0:
        b = vegas_odds / 100
    else:
        b = -100 / vegas_odds
    
    # Calculate the Kelly fraction
    q = 1 - probability
    kelly_fraction = (b * probability - q) / b
    
    # Calculate the bet size
    bet_size = kelly_fraction * bankroll
    
    return max(0, bet_size)  # Ensure the bet size is not negative

bankroll = 25  # $1,000 available for betting
vegas_odds = -125 # +150 odds
probability = 0.92940353# 55% chance of winning

bet_amount = kelly_criterion(bankroll, vegas_odds, probability)
print(f"You should bet ${bet_amount:.2f}")

You should bet $21.03


In [456]:
# NOTE!!!!!!!!!!!! THIS SHOULD ONLY BE USED ONCE TO CREATE A NEW CSV FOR A PLAYER. ONCE A CSV HAS BEEN CREATED, USE THE updateStats METHOD TO UPDATE THEIR MOST CURRENT GAME.

def importantStatsAccurate(player_name):
    player_gameLog = player_gamelog(player_name)
    player_gameLog = player_gameLog.iloc[::-1].reset_index(drop=True)
    playerID = player_id(player_name)
    player_gameLog['TEAM_ABBREVIATION'] = player_gameLog['MATCHUP'].apply(extract_team)
    player_gameLog['TEAM_ID'] = player_gameLog['TEAM_ABBREVIATION'].apply(team_id)
    player_gameLog['OPP_TEAM'] = player_gameLog['MATCHUP'].apply(extract_opponent)
    player_gameLog['OPP_TEAMID'] = player_gameLog['OPP_TEAM'].apply(team_id)
    homeOrAway(player_gameLog)
    player_gameLog['POSITION'] = player_gameLog['Player_ID'].apply(position)
    encoded_positions = player_gameLog['POSITION'].apply(encode_positions)
    player_gameLog = pd.concat([player_gameLog, encoded_positions], axis=1)
    player_gameLog['AGE'] = player_gameLog['Player_ID'].apply(age)
    player_gameLog['HEIGHT'] = player_gameLog['Player_ID'].apply(height)
    player_gameLog['WEIGHT'] = player_gameLog['Player_ID'].apply(weight)
    player_gameLog['TEAM_W'] = player_gameLog.apply(lambda row: getWinsUpTo(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['TEAM_L'] = player_gameLog.apply(lambda row: getLossesUpTo(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['PPG'] = player_gameLog.apply(lambda row: getPPGUpTo(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['L5_L'] = 5 - player_gameLog['L5_W']
    player_gameLog['L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['TEAM_ID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_TEAM_W'] = player_gameLog.apply(lambda row: getWinsUpTo(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_TEAM_L'] = player_gameLog.apply(lambda row: getLossesUpTo(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_PPG_ALLOWED'] = player_gameLog.apply(lambda row: getPPG_allowedUpTo(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_L5_W'] = player_gameLog.apply(lambda row: getL5Wins(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)
    player_gameLog['OPP_L5_L'] = 5 - player_gameLog['OPP_L5_W']
    player_gameLog['OPP_L5_PPG'] = player_gameLog.apply(lambda row: getL5PPG(row['OPP_TEAMID'], row['GAME_DATE']), axis=1)

    window_size = 5
    stats_to_average = ['PLUS_MINUS', 'MIN']
    for stat in stats_to_average:
        player_gameLog[f'MA_{stat}'] = player_gameLog[stat].rolling(window=window_size, min_periods=1).mean().shift(0)
        player_gameLog[f'AVG_{stat}'] = player_gameLog[stat].expanding().mean().shift(1)
    for stat in stats_to_average:
        player_gameLog.at[0, f'AVG_{stat}'] = player_gameLog.at[0, stat]

    stats_to_average = ['PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'STL', 'MIN']
    for stat in stats_to_average:
        player_gameLog[f'MA_{stat}'] = np.nan 
        player_gameLog[f'AVG_{stat}'] = np.nan
    for index, row in player_gameLog.iterrows():
        game_date = row['GAME_DATE']
        season_stats_df = seasonAverages(playerID, game_date)

        game_ids = last5ID(playerID, game_date)
        l5_stats_df = last5Stats(playerID, game_date) 

        if l5_stats_df.empty:
            for stat in stats_to_average:
                player_gameLog.at[index, f'AVG_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                player_gameLog.at[index, f'MA_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
        else:
            for stat in stats_to_average:
                player_gameLog.at[index, f'AVG_{stat}'] = season_stats_df[f'AVG_{stat}'].iloc[0]
                player_gameLog.at[index, f'MA_{stat}'] = l5_stats_df[f'AVG_{stat}'].iloc[0]

    player_gameLog['target'] = (player_gameLog['PTS'] > player_gameLog['AVG_PTS']).astype(int)

    player_gameLog.reset_index(drop=True, inplace=True)
    
    return player_gameLog

# playedList = playedYesterday()
# print(playedList)
players_list_all = [
    "Devin Booker", "Domantas Sabonis", "Donovan Mitchell", "Evan Mobley", "Giannis Antetokounmpo", "Jalen Brunson", "Jalen Green", "Jalen Johnson", "Jalen Williams", "Jamal Murray", "James Harden", "Jaren Jackson Jr.", "Jaylen Brown", "Jayson Tatum", "John Collins", "Jordan Poole", "Julius Randle", "Karl-Anthony Towns", "Kevin Durant", "Kristaps Porziņģis", "Kyrie Irving", "LaMelo Ball", "Lauri Markkanen", "LeBron James", "Luka Dončić", "Malik Monk", "Michael Porter Jr.", "Mikal Bridges", "Miles Bridges", "Nikola Jokić", "Nikola Vučević", "Norman Powell", "Pascal Siakam", "Paul George", "RJ Barrett", "Shaedon Sharpe", "Shai Gilgeous-Alexander", "Stephen Curry", "Trae Young", "Tyler Herro", "Tyrese Haliburton", "Tyrese Maxey", "Victor Wembanyama", "Zach LaVine",

    "CJ McCollum", "Zion Williamson", "De'Andre Hunter", "Andrew Wiggins", "Johnathan Kuminga", "Keyonte George", "Bennedict Mathurin", "Jalen Suggs", "Malik Beasley", "Gradey Dick", "OG Anunoby", "Jimmy Butler", "Derrick White", "Bam Adebayo", "Myles Turner", "Fred VanVleet", "Jerami Grant", "Ivica Zubac", "Dennis Schröder", "Jakob Poeltl", "Payton Pritchard", "Deni Avdija", "Christian Braun", "Josh Hart", "Jarrett Allen", "Deandre Ayton", "P.J. Washington", "Kelly Oubre Jr.", "Naz Reid", "Klay Thompson", "Dyson Daniels", "Bobby Portis", "Russell Westbrook", "Tobias Harris", "Amen Thompson", "D'Angelo Russell", "Dillon Brooks", "Santi Aldama", "Ayo Dosunmu", "Bilal Coulibaly", "Keldon Johnson", "Brook Lopez", "Daniel Gafford", "Rui Hachimura", "Buddy Hield", "Scoot Henderson", "Jabari Smith Jr.", "Keegan Murray", "Ty Jerome"
]
for player in players_list_all:
    file_path = os.path.join('v2.4 csvs', player + '.csv')
    df_updated = importantStatsAccurate(player)
    df_updated.to_csv(file_path)
    df_updated.to_excel('SGA.xlsx')
    print("This player's csv was created: ", player)
masterTESTcsv = concatAllCSVS('v2.4 csvs')
masterTESTcsv.to_csv('masterCSV_2025-01-26 v2.4.csv')

KeyboardInterrupt: 

In [130]:
# THIS IS TO CREATE A MODEL WITH EVERYONE'S CSVS CONCATTED
def createModelALL():
    file_path = 'masterCSV_2025-01-25 v2.3.csv'  # Specify your CSV file name here
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)

    # Features including PLAYER_ID
    categorical_features = ['MATCHUP', 'Player_ID', 'TEAM_ID', 'OPP_TEAMID']
    classifier = Pipeline([
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
            remainder='passthrough')),
        ('classifier', LogisticRegression(solver = 'liblinear', max_iter=50000, warm_start=True))
    ])
    # 'TEAM_ID', 'OPP_TEAMID',
    X = df[['Player_ID','MATCHUP','TEAM_ID','OPP_TEAMID','Home_Away','Primary_Guard','Primary_Forward','Primary_Center','Is_Guard','Is_Forward','Is_Center','AGE','HEIGHT','WEIGHT','TEAM_W','TEAM_L','PPG','L5_W','L5_L','L5_PPG','OPP_TEAM_W','OPP_TEAM_L','OPP_PPG_ALLOWED','OPP_L5_W','OPP_L5_L','OPP_L5_PPG','MA_PTS','MA_FGM','MA_FGA','MA_FG_PCT','MA_FG3M','MA_FG3A','MA_FG3_PCT','MA_OREB','MA_STL','MA_MIN','MA_PLUS_MINUS','AVG_PTS','AVG_FGM','AVG_FGA','AVG_FG_PCT','AVG_FG3M','AVG_FG3A','AVG_FG3_PCT','AVG_FTM','AVG_FTA','AVG_FT_PCT','AVG_OREB','AVG_STL','AVG_MIN','AVG_PLUS_MINUS','VS_AVG_PTS','VS_AVG_FGM','VS_AVG_FGA','VS_AVG_FG_PCT','VS_AVG_FG3M','VS_AVG_FG3A','VS_AVG_FG3_PCT','VS_AVG_OREB','VS_AVG_STL','VS_AVG_MIN','VS_AVG_PLUS_MINUS']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    # Save the model
    joblib.dump(classifier, 'master_model_lib v2.3.pkl')
createModelALL()

Accuracy: 0.6586102719033232
              precision    recall  f1-score   support

           0       0.61      0.70      0.65       150
           1       0.72      0.62      0.67       181

    accuracy                           0.66       331
   macro avg       0.66      0.66      0.66       331
weighted avg       0.67      0.66      0.66       331

